Import Libraries

In [165]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import math 
import cv2
import pytesseract
import re
from time import time
from decimal import ROUND_UP
from math import ceil
import pandas as pd
import sys, os
import streamlit as st

User Inputs

In [8]:
#application_path = os.path.dirname(sys.executable)

URL = input('URL: ')
# userMaxPrice = input('Enter Maximum Price: ')
# priceIterator = input('Price Iterator: ')
# minBedNo = input('Minimum number of bedrooms: ')

userMaxPrice = '700000'
priceIterator = '50000'
minBedNo = '2'

# splits the URL to the relevant area only and ignores all the features selected on the website
beforeURL = URL.split('%')[0] + '%'
afterURL = URL.split('%')[1]
URL = afterURL.split('&')[0]
generatedURL = beforeURL + URL + '&minBedrooms=' + minBedNo
print(generatedURL)

https://www.rightmove.co.uk/property-for-sale/find.html?searchType=SALE&locationIdentifier=REGION%5E1261&minBedrooms=2


Get all the property addresses

In [9]:
# rightmove main URL
### initialise search ### 

allLinks = np.array([])
allLinksCombined = np.array([])
cleanLinks = np.array([])
minPrice = 50000
maxPrice = minPrice + int(priceIterator)

pageIterator = (int(userMaxPrice)-minPrice)/int(priceIterator)
for x in range(int(pageIterator)):
    URL = generatedURL + '&maxPrice=' + str(maxPrice) + '&minPrice=' + str(minPrice)
    page = requests.get(URL)

    # parse search results card
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id='l-container')

    # index for iterator (remember each page has 24 ads)
    noOfResults = soup.find('div', attrs={'id': 'searchHeader'})
    noOfResults = noOfResults.select('span')[0].text
    noOfResults = re.sub(r',', '', noOfResults)
    noOfResults = ceil(int(noOfResults)/24) # round up to nearest page (note max page no is 42)

    print(noOfResults, " Pages available for min price = ", minPrice, " max price = ", maxPrice)

    i = 0
    all_ids = np.array([])
    for x in range(noOfResults):
        URL = generatedURL + '&maxPrice=' + str(maxPrice) + '&minPrice=' + str(minPrice) + '&sortType=18&index='+ str(i)
        print('Scrapped Page No.', x+1)
        page = requests.get(URL)

        # parse search results card
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id='l-container')

        # Extract and store ids
        links = results.select('a')
        for card in links:
            ids = card.get('id')
            ids = ids.strip() if ids is not None else ''
            all_ids = np.append(all_ids, ids)

        # clean array and add links
        all_ids = all_ids[all_ids != '']
        cleanIds = np.char.strip(all_ids, 'prop')
        allLinks = np.char.add(cleanIds, '#/?channel=RES_BUY')
        allLinks = np.char.add('https://www.rightmove.co.uk/properties/', allLinks)
    
        i = i + 24

    allLinksCombined = np.append(allLinksCombined, allLinks)
    minPrice = minPrice + int(priceIterator) 
    maxPrice = maxPrice + int(priceIterator) 

# removes empty and repeated links 
cleanLinks = np.char.replace(allLinksCombined, 'https://www.rightmove.co.uk/properties/0#/?channel=RES_BUY', '')
cleanLinks = cleanLinks[cleanLinks != '']
cleanLinks = np.unique(cleanLinks)

print('Number of links acquired: ', len(cleanLinks))
print(cleanLinks)


0  Pages available for min price =  50000  max price =  100000
0  Pages available for min price =  100000  max price =  150000
0  Pages available for min price =  150000  max price =  200000
0  Pages available for min price =  200000  max price =  250000
1  Pages available for min price =  250000  max price =  300000
Scrapped Page No. 1
1  Pages available for min price =  300000  max price =  350000
Scrapped Page No. 1
1  Pages available for min price =  350000  max price =  400000
Scrapped Page No. 1
2  Pages available for min price =  400000  max price =  450000
Scrapped Page No. 1
Scrapped Page No. 2
1  Pages available for min price =  450000  max price =  500000
Scrapped Page No. 1
1  Pages available for min price =  500000  max price =  550000
Scrapped Page No. 1
2  Pages available for min price =  550000  max price =  600000
Scrapped Page No. 1
Scrapped Page No. 2
1  Pages available for min price =  600000  max price =  650000
Scrapped Page No. 1
1  Pages available for min price 

Fishing Properties

In [10]:
i = 0
counter = 0
all_info = np.array([])
data = np.array([])

for property in cleanLinks:
    start = time()
    ad = requests.get(property)
    soup = BeautifulSoup(ad.content, "html.parser")

    # ensures only freehold properties
    if soup.select('p')[4].text.lower() == 'freehold' or soup.select('p')[6].text.lower() == 'freehold':

        # extract property data
        propertyAddress = soup.select('h1')[0].text
        propertyType = soup.select('p')[0].text
        typeOfProperty = soup.select('p')[1].text
        noOfBeds = soup.select('p')[2].text
        priceLink = soup.body.find('div', attrs={'class': '_1gfnqJ3Vtd1z40MlC0MzXu'})
        price = priceLink.select('span')[0].text
        adDateLink = soup.body.find('div', attrs={'class': '_2nk2x6QhNB1UrxdI5KpvaF'})
        date = re.sub('<div class="_2nk2x6QhNB1UrxdI5KpvaF">', '', str(adDateLink))
        date = re.sub('</div>', '', str(date))
        newIds = re.findall(r'\b\d+\b', str(cleanLinks[i])) # use new property id
        mapLocation = np.char.add('https://www.rightmove.co.uk/properties/', str(newIds[0]))
        floorPlan = np.char.add(mapLocation, '#/floorplan?activePlan=1&channel=RES_BUY')
        mapLocation = np.char.add(mapLocation, '#/map?channel=RES_BUY')
        
        # filter through the images to find the floor plan link
        floorPlanImage = requests.get(floorPlan)
        floorPlanImage = BeautifulSoup(floorPlanImage.content, "html.parser")
        floorPlanImage = floorPlanImage.find('div', attrs={'id': 'root'})
        for j in range(len(floorPlanImage.select('img'))):
            nextFloorPlanImage = floorPlanImage.select('img')[j].get_attribute_list('src',str)
            nextFloorPlanImage = ' '.join(nextFloorPlanImage)
            if '_max_296x197' in nextFloorPlanImage:
                foundFloorPlanImage = nextFloorPlanImage.replace('_max_296x197', '')
                break
            else:
                foundFloorPlanImage = 'None'
                pass
        
        i = i+1
        end = time()
        print(str(i)+' out of '+str(len(cleanLinks))+f' It took {round(end - start,2)} seconds!') 

        data = [str(price), str(propertyAddress), str(typeOfProperty), str(noOfBeds), str(foundFloorPlanImage), str(mapLocation), str(date)]
        all_info = np.append(all_info, data)
        counter = counter + 1
    else:
        #print('skipped...')
        i = i+1
        end = time()
        print(str(i)+' out of '+str(len(cleanLinks))+f' It took {round(end - start,2)} seconds!') 
        continue

# split the array into equally divided arrays
all_info = np.split(all_info, counter)
print(all_info)

1 out of 107 It took 0.45 seconds!
2 out of 107 It took 0.85 seconds!
3 out of 107 It took 0.45 seconds!
4 out of 107 It took 0.55 seconds!
5 out of 107 It took 0.49 seconds!
6 out of 107 It took 0.92 seconds!
7 out of 107 It took 0.93 seconds!
8 out of 107 It took 0.72 seconds!
9 out of 107 It took 0.9 seconds!
10 out of 107 It took 0.45 seconds!
11 out of 107 It took 0.43 seconds!
12 out of 107 It took 1.92 seconds!
13 out of 107 It took 0.89 seconds!
14 out of 107 It took 0.67 seconds!
15 out of 107 It took 0.93 seconds!
16 out of 107 It took 0.46 seconds!
17 out of 107 It took 1.03 seconds!
18 out of 107 It took 0.88 seconds!
19 out of 107 It took 0.42 seconds!
20 out of 107 It took 0.63 seconds!
21 out of 107 It took 0.39 seconds!
22 out of 107 It took 0.42 seconds!
23 out of 107 It took 0.44 seconds!
24 out of 107 It took 0.44 seconds!
25 out of 107 It took 0.43 seconds!
26 out of 107 It took 0.5 seconds!
27 out of 107 It took 0.42 seconds!
28 out of 107 It took 0.41 seconds!
29 

Image Processing

In [161]:
import importlib 
from numpy import empty
import searchFunction
importlib.reload(searchFunction) # forces the definition to reload in case you make changes to it
from searchFunction import search 

updated_info = np.array([])

for info in range(10):#len(all_info)):
    print(str(info+1)+' out of '+str(len(all_info)))
    try:
        if not all_info[info][4] == 'None':
            # Load the img
            req = urllib.request.urlopen(all_info[info][4])
            arr = np.asarray(bytearray(req.read()))
            img = cv2.imdecode(arr, -1) # 'Load it as it is'

            # Display loaded image
            # cv2.imshow('Floor Plan', img)
            # if cv2.waitKey() & 0xff == 27: quit()

            # OCR
            pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
            configs = r'--psm 6'
            try:
                rawText = pytesseract.image_to_string(img, config=configs)
            except:
                continue
            rawText = rawText.lower()
            rawText = re.sub(r',', '', str(rawText)) # removes all commas 
            rawText = re.sub(r'\b\.\d+\b', '', str(rawText)) # removes all ints
            #print(rawText)

            newText = search(rawText)
            #print(newText)
            newInfo = list(np.append(all_info[info], newText)) # adding list adds the ',' back into the string array    
            updated_info = np.append(updated_info, newInfo)
    except:
        continue        

# split the array into equally divided arrays and convert to pandas for further processing
updated_info = np.split(updated_info, len(updated_info)/8) 
updated_info = pd.DataFrame(updated_info, columns = ['price', 'propertyAddress', 'typeOfProperty', 'noOfBeds', 'foundFloorPlanImage', 'mapLocation', 'date', 'area'])
updated_info = updated_info.drop(columns='foundFloorPlanImage')

# Tidy data up
updated_info['noOfBeds'] = updated_info['noOfBeds'].str[1:]
date = pd.DataFrame(updated_info['date'].str.split('on', expand=True))
date.columns = ['status', 'date']
updated_info = updated_info.drop(columns='date')
updated_info = updated_info.join(date)
print(updated_info)

1 out of 34
2 out of 34
3 out of 34
4 out of 34
5 out of 34
6 out of 34
7 out of 34
8 out of 34
9 out of 34
10 out of 34
      price                          propertyAddress       typeOfProperty  \
0  £595,000            Daventer Drive, Stanmore, HA7  Link Detached House   
1  £565,000  Flecker Close, Stanmore, Middlesex, HA7             Terraced   
2  £650,000         St. Edmunds Drive, Stanmore, HA7        Semi-Detached   
3  £540,000    Taunton Way, Stanmore, Middlesex, HA7        Semi-Detached   
4  £585,000                 Stanmore, Middlesex, HA7        Semi-Detached   
5  £650,000               Eaton Close, Stanmore, HA7             Terraced   
6  £600,000           Brightwen Grove, Stanmore, HA7        Semi-Detached   
7  £695,000            Merrion Avenue, Stanmore, HA7        Semi-Detached   
8  £565,000          Castellane Close, Stanmore, HA7  Link Detached House   

  noOfBeds                                        mapLocation  area    status  \
0        3  https://www.rig

In [164]:
# Beginning of CrystalRoof 

# propertyAddress = updated_info['propertyAddress'] 
# print(propertyAddress)

0              Daventer Drive, Stanmore, HA7
1    Flecker Close, Stanmore, Middlesex, HA7
2           St. Edmunds Drive, Stanmore, HA7
3      Taunton Way, Stanmore, Middlesex, HA7
4                   Stanmore, Middlesex, HA7
5                 Eaton Close, Stanmore, HA7
6             Brightwen Grove, Stanmore, HA7
7              Merrion Avenue, Stanmore, HA7
8            Castellane Close, Stanmore, HA7
Name: propertyAddress, dtype: object


In [166]:
# Streamlit Code
st.title('RightMove Scrapper')

2023-01-14 00:49:48.655 
  command:

    streamlit run c:\Users\Hesha\Documents\GitHub\Python-Codes\rightMove Scrapper\.venv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [162]:
# this should load the old file and append any new data found to it making sure it's all unique data
# and not repeated 

updated_info.to_csv('output.csv', index=False) # comment out if you want to use the old data

loadData = pd.read_csv(r'output.csv')
frames = [loadData, updated_info]
combinedData = pd.concat(frames)
combinedData = combinedData.drop_duplicates(subset=['mapLocation'])

combinedData['price'] = combinedData['price'].replace('[\£,]', '', regex=True).astype(float)
combinedData['area'] = combinedData['area'].astype(float)
combinedData['priceRank'] = combinedData['price'].rank(ascending=1)
combinedData['areaRank'] = combinedData['area'].rank(ascending=0)
combinedData['Rank Sum'] = combinedData.priceRank + combinedData.areaRank
combinedData = combinedData.sort_values(by = 'Rank Sum', ascending = True)

combinedData.to_csv('output_processed.csv', index=False)
print(combinedData)

      price                          propertyAddress       typeOfProperty  \
1  565000.0  Flecker Close, Stanmore, Middlesex, HA7             Terraced   
4  585000.0                 Stanmore, Middlesex, HA7        Semi-Detached   
2  650000.0         St. Edmunds Drive, Stanmore, HA7        Semi-Detached   
8  565000.0          Castellane Close, Stanmore, HA7  Link Detached House   
3  540000.0    Taunton Way, Stanmore, Middlesex, HA7        Semi-Detached   
0  595000.0            Daventer Drive, Stanmore, HA7  Link Detached House   
5  650000.0               Eaton Close, Stanmore, HA7             Terraced   
6  600000.0           Brightwen Grove, Stanmore, HA7        Semi-Detached   
7  695000.0            Merrion Avenue, Stanmore, HA7        Semi-Detached   

  noOfBeds                                        mapLocation    area  \
1        4  https://www.rightmove.co.uk/properties/1242386...  1305.0   
4        3  https://www.rightmove.co.uk/properties/1258245...  1111.0   
2        5